For interacting with an AnnoRepo server, you can use the python library `annorepo-client`

If the AnnoRepo server requires authentication, you should have gotten a valid api-key.

In [2]:
from annorepo.client import AnnoRepoClient

annorepo_url='https://annorepo.dev.clariah.nl'
ar = AnnoRepoClient(annorepo_url, api_key="test-api-key")
ar.get_about()

{'appName': 'AnnoRepo',
 'version': '0.6.0',
 'startedAt': '2023-11-17T13:55:23.839020318Z',
 'baseURI': 'https://annorepo.dev.clariah.nl',
 'withAuthentication': True,
 'sourceCode': 'https://github.com/knaw-huc/annorepo',
 'mongoVersion': '6.0.3'}

First, we check what containers we have access to

In [3]:
ar.read_accessible_containers()

{}

Nothing yet, so let's create our first annotation container

In [4]:
(etag,location,body) = ar.create_container(name="my-test-container")
location

'https://annorepo.dev.clariah.nl/w3c/my-test-container/'

If we now check what containers we have access to, we see we have access rights to `my-test-container`

In [5]:
ar.read_accessible_containers()

{'ADMIN': ['my-test-container']}

We'll be working with this container now.

To save having to specify the container name in all our client calls, we generate a container_adapter for this container:

In [6]:
ca = ar.container_adapter("my-test-container")

Now we add our first annotation:

In [7]:
annotation1={
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "id": "http://example.org/anno1",
  "type": "Annotation",
  "body": "http://example.org/post1",
  "target": "http://example.com/page1"
}
ca.add_annotation(content=annotation1,name="basic-annotation-model")

{'@context': 'http://www.w3.org/ns/anno.jsonld',
 'id': 'https://annorepo.dev.clariah.nl/w3c/my-test-container/basic-annotation-model',
 'type': 'Annotation',
 'body': 'http://example.org/post1',
 'target': 'http://example.com/page1',
 'via': 'http://example.org/anno1'}

If we go to the url of the annotation in a browser, we see we don't have access to it without our api-key.

By default, a container and all its annotations are only accessible to logged-in users that have been added to the container users for that container.

There is an option to make the container read-only accessible for anonymous users (without an api-key):

In [8]:
ca.set_anonymous_user_read_access()

True

This succeeded, so now we can access the annotation url in the browser.

Now, let's add some more annotations, in one batch this time:

In [9]:
annotations = [
{
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "id": "http://example.org/anno2",
  "type": "Annotation",
  "body": {
    "id": "http://example.org/analysis1.mp3",
    "format": "audio/mpeg",
    "language": "fr"
  },
  "target": {
    "id": "http://example.gov/patent1.pdf",
    "format": "application/pdf",
    "language": ["en", "ar"],
    "textDirection": "ltr",
    "processingLanguage": "en"
  }
},
{
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "id": "http://example.org/anno3",
  "type": "Annotation",
  "body": {
    "id": "http://example.org/video1",
    "type": "Video"
  },
  "target": {
    "id": "http://example.org/website1",
    "type": "Text"
  }
},
{
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "id": "http://example.org/anno4",
  "type": "Annotation",
  "body": "http://example.org/description1",
  "target": {
    "id": "http://example.com/image1#xywh=100,100,300,300",
    "type": "Image",
    "format": "image/jpeg"
  }
},
{
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "id": "http://example.org/anno5",
  "type": "Annotation",
  "body": {
    "type" : "TextualBody",
    "value" : "<p>j'adore !</p>",
    "format" : "text/html",
    "language" : "fr"
  },
  "target": "http://example.org/photo1"
},
{
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "id": "http://example.org/anno6",
  "type": "Annotation",
  "bodyValue": "Comment text",
  "target": "http://example.org/target1"
},
{
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "id": "http://example.org/anno7",
  "type": "Annotation",
  "body": {
    "type": "TextualBody",
    "value": "Comment text",
    "format": "text/plain"
  },
  "target": "http://example.org/target1"
}
]
ca.add_annotations(annotation_list=annotations)

[{'annotationName': 'fa30efb3-2169-4223-86f2-143d8bf65dd8',
  'containerName': 'my-test-container',
  'etag': '809546646'},
 {'annotationName': 'fa01f59f-6c39-4c58-89e4-cdf6ac8a5aea',
  'containerName': 'my-test-container',
  'etag': '2087147021'},
 {'annotationName': 'd51e44b9-4be0-4734-9875-d13e45b35b58',
  'containerName': 'my-test-container',
  'etag': '829830413'},
 {'annotationName': '89f0a8fb-8a25-442c-abc8-6f1bc280a83f',
  'containerName': 'my-test-container',
  'etag': '939788185'},
 {'annotationName': 'c993c12b-0a8e-47c2-8fb6-4a286cb1c363',
  'containerName': 'my-test-container',
  'etag': '1567613796'},
 {'annotationName': 'a8d661b6-d731-4293-8ee1-5cc293af3681',
  'containerName': 'my-test-container',
  'etag': '1946132538'}]

Now, we can do a search, for example all annotations where the `format` of the `target` is 'application/pdf':

In [10]:
search_info = ca.create_search(query={'target.format':'application/pdf'})
search_info

SearchInfo(id='d7ea50b0-c68a-462b-b9a9-14a81a83ce9d', location='https://annorepo.dev.clariah.nl/services/my-test-container/search/d7ea50b0-c68a-462b-b9a9-14a81a83ce9d')

and we get the first (and, in this case, only) resultpage, and the annotations therein:

In [11]:
page = ca.read_search_result_page(search_info.id)
page['items']

[{'id': 'https://annorepo.dev.clariah.nl/w3c/my-test-container/fa30efb3-2169-4223-86f2-143d8bf65dd8',
  'type': 'Annotation',
  'body': {'id': 'http://example.org/analysis1.mp3',
   'format': 'audio/mpeg',
   'language': 'fr'},
  '@context': 'http://www.w3.org/ns/anno.jsonld',
  'target': {'id': 'http://example.gov/patent1.pdf',
   'format': 'application/pdf',
   'language': ['en', 'ar'],
   'textDirection': 'ltr',
   'processingLanguage': 'en'}}]